#Start session

In [ ]:
# ダウンロードのために必要なパッケージをインストール
!rm -f *.deb
# 足りなくなったら継ぎ足していく（今回はlibvulkan1)
!sudo apt install -y wget curl unzip libvulkan1
# 以下はChromeの依存パッケージ
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb

# Chromeのインストール
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb

# Chrome Driverのインストール
# 再実行した際に色々残っているとエラーになるので
!rm -rf /tmp/*
!wget -N 	https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.129/linux64/chromedriver-linux64.zip -P /tmp/

#!wget -N 	https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chromedriver-linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
!cp -rf /tmp/chromedriver-linux64 /tmp/chromedriver
!chmod +x /tmp/chromedriver/chromedriver
!mv /tmp/chromedriver/chromedriver /usr/local/bin/chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libvulkan1 is already the newest version (1.3.204.1-2).
wget is already the newest version (1.21.2-2ubuntu1).
curl is already the newest version (7.81.0-1ubuntu1.15).
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google-chrome.list:4
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google-chrome.list:4
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google-chrome.list:5
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.li

In [ ]:
! sudo sh -c 'echo "deb http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
! sudo wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
! sudo apt update
! sudo apt install google-chrome-stable

OK
Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (170 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
34 packages can be upgraded. Run 'apt list --upgrada

In [ ]:
#make sure versions match
!google-chrome --version
!chromedriver --version

Google Chrome 121.0.6167.184 
/bin/bash: line 1: chromedriver: command not found


In [ ]:
!pip install selenium
!pip install newspaper3k
!pip3 install news-please

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
from datetime import datetime
import time
import os
import re
import math
import pandas as pd
from newspaper import Article
from newsplease import NewsPlease
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from collections import Counter

from google.colab import files
import csv

In [ ]:
#test to see if can retrieve title
URL = "https://dev.classmethod.jp/"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    options=options
)

driver.get(URL)
title = driver.title
driver.close()

print(title)
#should show クラスメソッド発「やってみた」系技術メディア | DevelopersIO

クラスメソッド発「やってみた」系技術メディア | DevelopersIO


In [ ]:
from selenium.common.exceptions import NoSuchElementException, \
    WebDriverException

#Data Loader

In [ ]:
def load(path):
  data = pd.read_csv(path)
  return data

#Information Collector

In [ ]:
#def get_link_by_text(text):
#    """Find link in the page with given text"""
#    element = driver.find_element(By.LINK_TEXT,text.strip())
#    return element.get_attribute("href")

# def get_link_by_xpath(path):
#     """Find link in the page with given text"""
#     element = driver.find_element(By.XPATH,path)
#     return element.get_attribute("href")

def get_link(by, path):
    """Find link in the page with given text"""
    element = driver.find_element(by,path)
    return element.get_attribute("href")

def get_link_list(by, name):
    """Get list of text in all element by name"""
    element_list = []
    try:
        all_elements = driver.find_elements(by,name)
        element_list = [x.get_attribute('href') for x in all_elements
                        if len(x.get_attribute('href')) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

def get_list(by, name):
    """Get list of text in all element by name"""
    element_list = []
    try:
        all_elements = driver.find_elements(by,name)
        element_list = [x.text for x in all_elements if len(x.text) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

def get_onclick_list(by, name, root):
    """Get list of text in all element by name"""
    element_list = []
    try:
        all_elements = driver.find_elements(by,name)
        element_list = [root+x.get_attribute('onclick').split("'")[1]
                        for x in all_elements if len(x.text) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

def get_list2(by, name):
    """Get list of text in all element by name"""
    element_list = []
    try:
        all_elements = driver.find_elements(by,name)
        element_list = [x.get_attribute("innerHTML").splitlines()[1]
                        for x in all_elements if len(x.text) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

In [ ]:
def DriverSet():
    """Open Driver"""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(
      options=options
    )
    #driver.implicitly_wait(2)
    return driver

def Info(place):
  path=data.iloc[place]["Path"]
  titlepath=data.iloc[place]["TitlePath"] if str(data.iloc[place]["TitlePath"])!="nan" else 0
  next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
  i=int(data.iloc[place]["Next Page Start"]) if str(data.iloc[place]["Next Page Start"])!="nan" else 0
  by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
  nextbutton=data.iloc[place]["Next Button"] if str(data.iloc[place]["Next Button"])!="nan" else 0
  nextlink=data.iloc[place]["Next Page"] if str(data.iloc[place]["Next Page"])!="nan" else 0

  links = get_link_list(By.XPATH, path)  if str(data.iloc[place]["TitlePath"])!="nan" else 0
  titles = get_list(By.XPATH, titlepath)   if str(data.iloc[place]["TitlePath"])!="nan" else 0
  jump = 1 if str(data.iloc[place]["Checked?"])=="jump" else 0


In [ ]:
def textcollect(linklist):
  dfbase=[]
  for link in linklist:
    article=NewsPlease.from_url(link)
    #append information to dataframe
    dfbase.append([article.title,
                article.authors,
                article.date_publish,
                article.description,
                article.maintext,
                article.language,
                article.url])
  return dfbase

#Link Collector

In [ ]:
place=29

In [ ]:
#Individual Tester
data=load('/content/EngSpeech.csv')
driver=DriverSet()
driver.get(data.iloc[place]["Link"])
#Extract Necessary Info from CSV
paths=[data.iloc[place]["Path"]]
titlepath=data.iloc[place]["TitlePath"] if str(data.iloc[place]["TitlePath"])!="nan" else "0"

datepath="0"
if ",//" in str(paths[0]): #############################Edit rest of code so
#that we can go through all paths in same page and compile all links
  pathes=[]
  for i in paths[0].split(","):
    pathes.append(i)
  paths=pathes
if ",//" in str(titlepath):
  datepath=str(titlepath.split(",")[1])
  titlepath=str(titlepath.split(",")[0])

next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
i=int(data.iloc[place]["Next Page Start"]) if str(data.iloc[place]["Next Page Start"])!="nan" else 0
by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
nextbutton=data.iloc[place]["Next Button"] if str(data.iloc[place]["Next Button"])!="nan" else 0
nextlink=data.iloc[place]["Next Page"] if str(data.iloc[place]["Next Page"])!="nan" else "0"
nextlinkroot = str(data.iloc[place]["Next Page"])

if datepath!=0:
  dates = get_list(By.XPATH,datepath)

links=[]

for path in paths:
  links.extend(get_onclick_list(By.XPATH, path, nextlinkroot) if
               str(data.iloc[place]["Checked?"])=="split"
               else get_link_list(By.XPATH, path)
               if titlepath!="0" else str(0))


print(titlepath)
titles = get_list(By.XPATH, str(titlepath)) if     titlepath!="0" else 0

jump = 1 if str(data.iloc[place]["Checked?"])=="jump" else 0
click = 1 if str(data.iloc[place]["Checked?"])=="click" else 0
category_links = dict(map(lambda a,j,k: (a+" "+k,j) , titles,links,dates)) if (
    datepath) else dict(map(lambda a,j : (a,j) , titles,links)) if (
        titlepath!="0") else {
            x: get_link(By.LINK_TEXT,x.strip())
        for x in get_list(By.XPATH, path) for path in paths}
pages=int(data.iloc[place]["Pages"]) if (
    str(data.iloc[place]["Pages"])!="nan") else 0

print(category_links)

//tbody/tr/td[@class="   column-subject"]
{}


In [ ]:
if datepath!=0:
#  print("hi")
#  print(datepath)
#  jason=get_list(By.XPATH,'//tbody/tr/td[1]')
#  print(get_list(By.XPATH,datepath))
  dates = get_list(By.XPATH,datepath)
#  print(get_list(By.XPATH,datepath))

#dates = get_list(By.XPATH,datepath)
#print(dates)

links=[]

for path in paths:
  links.extend(get_onclick_list(By.XPATH, path, nextlinkroot) if
               str(data.iloc[place]["Checked?"])=="split"
               else get_link_list(By.XPATH, path)
               if titlepath!="0" else str(0))


print(titlepath)
titles = get_list(By.XPATH, str(titlepath)) if     titlepath!="0" else 0

jump = 1 if str(data.iloc[place]["Checked?"])=="jump" else 0
click = 1 if str(data.iloc[place]["Checked?"])=="click" else 0
category_links = dict(map(lambda a,j,k: (a+" "+k,j) , titles,links,dates)) if (
    datepath) else dict(map(lambda a,j : (a,j) , titles,links)) if (
        titlepath!="0") else {
            x: get_link(By.LINK_TEXT,x.strip())
        for x in get_list(By.XPATH, path) for path in paths}
pages=int(data.iloc[place]["Pages"]) if (
    str(data.iloc[place]["Pages"])!="nan") else 0

print(category_links)

//tbody/tr/td[@class="   column-subject"]
{}


In [ ]:
titles

[]

In [ ]:
#Individual Tester
data=load('/content/EngSpeech.csv')
driver=DriverSet()
driver.get(data.iloc[place]["Link"])
path=data.iloc[place]["Path"]
#path='//div/div/header/h3[@class="entry-title"]/a'

titlepath=data.iloc[place]["TitlePath"] if str(
    data.iloc[place]["TitlePath"])!="nan" else 0
#titlepath='//div/div[@class="dmarchive_results_title"]/a'
next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
i=int(data.iloc[place]["Next Page Start"]) if str(
    data.iloc[place]["Next Page Start"])!="nan" else 0
by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
nextbutton=data.iloc[place]["Next Button"] if str(
    data.iloc[place]["Next Button"])!="nan" else 0

nextlinkroot = str(data.iloc[place]["Next Page"])

links = get_onclick_list(By.XPATH, path, nextlinkroot) if str(
    data.iloc[place]["Checked?"])=="split" else get_link_list(
        By.XPATH, path)  if titlepath!=0 else 0
titles = get_list(By.XPATH, titlepath)   if str(
    data.iloc[place]["TitlePath"])!="nan" else 0
category_links = dict(map(lambda a,j: (a,j) , titles,links)) if (
    titlepath!=0) else {x: get_link(By.LINK_TEXT,x.strip()) for
                        x in get_list(By.XPATH, path)}
print(category_links)

#driver.close()

Message: invalid selector: Unable to locate an element with the xpath expression //tbody/tr/td[@class="   column-subject"],//tbody/tr/td[1] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//tbody/tr/td[@class="   column-subject"],//tbody/tr/td[1]' is not a valid XPath expression.
  (Session info: chrome-headless-shell=121.0.6167.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
#0 0x5929bbd17d93 <unknown>
#1 0x5929bb9fb337 <unknown>
#2 0x5929bba0069a <unknown>
#3 0x5929bba0296f <unknown>
#4 0x5929bba02a1c <unknown>
#5 0x5929bba46a6e <unknown>
#6 0x5929bba46e51 <unknown>
#7 0x5929bba8ac01 <unknown>
#8 0x5929bba6946d <unknown>
#9 0x5929bba8812d <unknown>
#10 0x5929bba691e3 <unknown>
#11 0x5929bba39135 <unknown>
#12 0x5929bba3a13e <unknown>
#13 0x5929bbcdbe4b <unknown>
#14 0x5929bbcdfdfa <unknown>
#15 0x5929bbcc86d5 <unknown

In [ ]:
#Tester for all at once

#Load the Data
data=load('/content/EngSpeech.csv')

place=28

#Prepare Driver
driver=DriverSet()

#Access Link
url=data.iloc[place]["Link"]
driver.get(url)

#Extract Necessary Info from CSV
paths=[data.iloc[place]["Path"]]
titlepath=data.iloc[place]["TitlePath"] if (
    str(data.iloc[place]["TitlePath"])!="nan") else 0

datepath=0
if ",//" in str(paths[0]): #############################Edit rest of code so
#that we can go through all paths in same page and compile all links
  pathes=[]
  for i in paths[0].split(","):
    pathes.append(i)
  paths=pathes
if ",//" in str(titlepath):
  datepath=titlepath.split(",")[1]
  titlepath=titlepath.split(",")[0]

next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
i=int(data.iloc[place]["Next Page Start"]) if (
    str(data.iloc[place]["Next Page Start"])!="nan") else 0
by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
nextbutton=data.iloc[place]["Next Button"] if (
    str(data.iloc[place]["Next Button"])!="nan") else 0
nextlink=data.iloc[place]["Next Page"] if (
    str(data.iloc[place]["Next Page"])!="nan") else "0"
nextlinkroot = str(data.iloc[place]["Next Page"])

if datepath!=0:
  dates = get_list(By.XPATH, datepath)
links=[]
for path in paths:
  links.extend(get_onclick_list(By.XPATH, path, nextlinkroot) if
               str(data.iloc[place]["Checked?"])=="split"
               else get_link_list(By.XPATH, path)
               if titlepath!="0" else str(0))
titles = get_list(By.XPATH, titlepath) if (
    str(data.iloc[place]["TitlePath"])!="nan") else 0
jump = 1 if str(data.iloc[place]["Checked?"])=="jump" else 0
click = 1 if str(data.iloc[place]["Checked?"])=="click" else 0
category_links = dict(map(lambda a,j,k: (a+" "+k,j) , titles,links,dates)) if (
    datepath) else dict(map(lambda a,j : (a,j) , titles,links)) if (
        str(data.iloc[place]["TitlePath"])!="nan") else {
            x: get_link(By.LINK_TEXT,x.strip())
        for x in get_list(By.XPATH, path) for path in paths}
pages=int(data.iloc[place]["Pages"]) if (
    str(data.iloc[place]["Pages"])!="nan") else 0

#Now we start collecting links
secondpage=True
while True:

  #If link is unresponsive, break
  if str(data.iloc[place]["Checked?"])=="Unresponsive":
    break

  #Collect links from the Second Page
  if secondpage==True:
    if jump!=1 and nextlink!="0" and str(nextbutton)=="0": #Go to 2nd page
    # if regular type and click!=1
      print("2nd page"+" "+f'{nextlink}{i}')
      driver.get(f'{nextlink}{i}')
      category_links2 = dict(map(lambda a,j : (a,j) , titles,links)) if (
          str(data.iloc[place]["TitlePath"])!="nan") else {
              x: get_link(By.LINK_TEXT,x.strip()) for x in get_list(By.XPATH, path)}
      category_links.update(category_links2)
    secondpage=False #Now never 2nd page

  #See if the next page exists
  try:

    #If jump, find the next button and collect the link to go to next page
    if jump==1:
      print("hi")
      nexturl= get_link(By.XPATH,nextbutton)#next link

    #If we have NEXT TEXT, PAGE LIMITS, NEXT BUTTON PATH, or the NEXT LINK
    # ROOT, then do the following
    elif next!=0 or pages!=0 or nextbutton!=0 or nextlink!="0":
      print('hi3')
      #If we have NEXT TEXT, find the button w the text and collect the link
      #to go to next page
      if str(next)!="0":
        print('hey1')
        nexturl= driver.find_element(By.PARTIAL_LINK_TEXT,next
                                     ).get_attribute("href")

      #If we have NEXT BUTTON PATH, then do the following:
      elif str(nextbutton)!="0":
        print('hey2')
        #If we have NEXT BUTTON PATH and the NEXT LINK ROOT, then:
        if nextlink!="0":
          print('hello1')
          #If we have NXT BTTN PTH nd NXT LNK RT and PAGE INCREASE RATE, then
          #set the next url as the next link root/page
          if by!=0:
            print("heehee")
            nexturl= (f'{nextlink}{i}')
          #If we just have the NXT BTTN PTH nd NXT LNK RT, then get the link
          # to next page by NEXT BUTTON PATH
          else:
            print("hoho")
            nexturl= get_link(By.XPATH,nextbutton)
        #If we have NEXT BUTTON PATH but NO NEXT LINK ROOT, then
        else:
          print('hello2')
          try:
            driver.find_element(By.XPATH,nextbutton).click()
            nexturl="0"
          except ElementNotInteractableException or NoSuchElementException:
            print("No next page!")
            break

      #If we have NEXT LINK ROOT, then do the following:
      elif nextlink!="0":
        print('hey3')
        #If we have NEXT LINK ROOT and PAGE LIMIT, then
        if pages!=0:
          nexturl= (f'{nextlink}{i}')
        #If we have NEXT LINK ROOT, but no NEXT TEXT, NOR
        # PAGE INCREASE RATE, NOR CURRENT PAGINATION PLACE, then
        elif next==0 and i==0 and by==0:
          print('hello1')
          print("No next page")
          break

        else:
          print("Error")
          break#######################FIXXXXXX

    #Otherwise we have no way of finding next page so break
    else:
      print('hi4')
      print("No next page")
      break

    i+=by if by!=0 else 1


  #If the Next Page does not Exist, break
  except NoSuchElementException:
    print("No next page!")
    break

  #If the Next Page Exists, then get the links
  else:
    print(i)
    print("h")
    #If not click, access the next page link
    if nexturl!="0":
      driver.get(nexturl)
      print(nexturl)
    time.sleep(5)
    links = get_link_list(By.XPATH, path)  if str(
        data.iloc[place]["TitlePath"])!="nan" else 0
    titles = get_list(By.XPATH, titlepath)   if str(
        data.iloc[place]["TitlePath"])!="nan" else 0
    category_links2 = dict(map(lambda a,j : (a,j) , titles,links)) if str(
        data.iloc[place]["TitlePath"])!="nan" else {
            x: get_link(By.LINK_TEXT,x.strip()) for x in get_list(By.XPATH, path)}
    category_links.update(category_links2)
    if i>=pages and pages!=0:
      break

    #Get rid of this when ready
    #if i==5:
    #  break

2nd page https://boj.org.jm/news/boj-speeches/2
hi3
hey3
3
h
https://boj.org.jm/news/boj-speeches/2


In [ ]:
category_links

{'Governor’s Monetary Policy Press Statement – November 2023': 'https://boj.org.jm/governors-monetary-policy-press-statement-november-2023/',
 'Governor’s Monetary Policy Press Statement – August 2023': 'https://boj.org.jm/governors-monetary-policy-press-statement-august-2023/',
 'Governor’s Monetary Policy Press Statement – May 2023': 'https://boj.org.jm/governors-monetary-policy-press-statement-may-2023/',
 'Governor’s Monetary Policy Press Statement – Feb 2023': 'https://boj.org.jm/governors-monetary-policy-press-statement-feb-2023/',
 'Governor Byles’ Speaking Notes: Standing Finance Committee Meeting -14 December 2022': 'https://boj.org.jm/governor-byles-speaking-notes-standing-finance-committee-meeting-14-dec2022/',
 'Opening Remarks by Governor Richard Byles on New Banknotes – December 2022': 'https://boj.org.jm/opening-remarks-by-governor-richard-byles-on-new-banknotes-december-2022/',
 'Governor’s Monetary Policy Press Statement – Nov 2022': 'https://boj.org.jm/governors-monet

In [ ]:
driver.close()

#Now Collect Info


In [ ]:
data=load('/content/EngSpeech.csv')

for place in range(len(data)):

  driver=DriverSet()

  doctype=data.iloc[place]["Doc Type"]
  Pais=data.iloc[place]["Pais"]
  Type=data.iloc[place]["Type"]
  url=data.iloc[place]["Link"]

  if doctype=="Article" or doctype=="Mixed":
      driver.get(url)
      linkscollected=collector(whatever)#need to fix this part

      dfbase=textcollect(linkscollected)

      df=pd.DataFrame(dfbase,columns=["Title","Authors","Date Published","Description","Main Text","Language","URL"])
      df.to_csv(f'{Pais}_{Type}_{doctype}.csv',sep=",",quotechar='"',index=False,quoting=csv.QUOTE_ALL)

      #df.to_csv('IDBTT01.csv')
      files.download(f'{Pais}_{Type}_{doctype}.csv')


In [ ]:
article=NewsPlease.from_url("https://www.ttparliament.org/wp-content/uploads/2024/01/hh20240126u.pdf")

In [ ]:
[article.title,
               article.authors,
               article.date_publish,
               article.description,
               article.maintext,
               article.language,
               article.url]

[None,
 [],
 None,
 None,
 None,
 'en',
 'https://www.ttparliament.org/wp-content/uploads/2024/01/hh20240126u.pdf']

#Tester

In [ ]:
for place in range(len(data)):
  options = webdriver.ChromeOptions()
  options.add_argument("--headless")
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument("--no-sandbox")

  driver = webdriver.Chrome(
      options=options
  )

  driver.get(data.iloc[place]["Link"])
  path=data.iloc[place]["Path"]

  titlepath=data.iloc[place]["TitlePath"] if str(data.iloc[place]["TitlePath"])!="nan" else 0
  next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
  i=int(data.iloc[place]["Next Page Start"]) if str(data.iloc[place]["Next Page Start"])!="nan" else 0
  by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
  nextbutton=data.iloc[place]["Next Button"] if str(data.iloc[place]["Next Button"])!="nan" else 0
  links = get_link_list(By.XPATH, path)  if str(data.iloc[place]["TitlePath"])!="nan" else 0
  titles = get_list(By.XPATH, titlepath)   if str(data.iloc[place]["TitlePath"])!="nan" else 0
  category_links = dict(map(lambda a,j : (a,j) , titles,links)) if str(data.iloc[place]["TitlePath"])!="nan" else {x: get_link_by_text(x) for x in get_list(By.XPATH, path)}
  print(category_links)

  driver.close()

{'Tancoo: calls on Imbert to have a heart and indemnify the population': 'https://unctt.org/tancoo-calls-on-imbert-to-have-a-heart-and-indemnify-the-population/', 'Rowley’s Mad Power Grab in Tobago': 'https://unctt.org/rowleys-mad-power-grab-in-tobago/', 'Presentation of UNC general election candidates 2020': 'https://unctt.org/presentation-of-unc-general-election-candidates-2020/', 'Kamla: My vision is to build an intelligent nation': 'https://unctt.org/kamla-my-vision-is-to-build-an-intelligent-nation/', 'MNF – Kamla: There is hope': 'https://unctt.org/mnf-kamla-there-is-hope/', 'Political Leader Speech at UNC National Congress 2020': 'https://unctt.org/political-leader-speech-at-unc-national-congress-2020/', 'Political Leader Address at UNC’S Local Government Public Meeting, Grande': 'https://unctt.org/political-leader-address-at-uncs-local-government-public-meeting-grande/', 'Kamla: The UNC is not about talk': 'https://unctt.org/kamla-the-unc-is-not-about-talk/', 'Opposition Leader

In [ ]:
data=load('/content/EngSpeech.csv')
place=28
driver=DriverSet()

url=data.iloc[place]["Link"]
driver.get(url)

element = driver.find_element(By.XPATH,'//span[text()="Load More"]')
#driver.execute_script("arguments[0].scrollIntoView();", element)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
  try:
    element = driver.find_element(By.XPATH,'//*[@id="post-20648"]/div/div/div/section[1]/div/div/div/div/div/div[3]')
    # Scroll to "Load More".
    #driver.execute_script("arguments[0].scrollIntoView();", element)

    element.click()
  # Wait to load the page.
    time.sleep(5)

  # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
      break

    last_height = new_height
  except NoSuchElementException: #If not then
    print("No next page!")
    break


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome-headless-shell=121.0.6167.160)
Stacktrace:
#0 0x597aa11d0d93 <unknown>
#1 0x597aa0eb4177 <unknown>
#2 0x597aa0f00fba <unknown>
#3 0x597aa0ef46b7 <unknown>
#4 0x597aa0f22442 <unknown>
#5 0x597aa0ef4028 <unknown>
#6 0x597aa0f2260e <unknown>
#7 0x597aa0f4112d <unknown>
#8 0x597aa0f221e3 <unknown>
#9 0x597aa0ef2135 <unknown>
#10 0x597aa0ef313e <unknown>
#11 0x597aa1194e4b <unknown>
#12 0x597aa1198dfa <unknown>
#13 0x597aa11816d5 <unknown>
#14 0x597aa1199a6f <unknown>
#15 0x597aa116569f <unknown>
#16 0x597aa11be098 <unknown>
#17 0x597aa11be262 <unknown>
#18 0x597aa11cff34 <unknown>
#19 0x78936ca7cac3 <unknown>


In [ ]:
driver.find_element(By.XPATH,'//*[@id="post-20648"]/div/div/div/section[1]/div/div/div/div/div/div[3]').click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome-headless-shell=121.0.6167.160)
Stacktrace:
#0 0x597aa11d0d93 <unknown>
#1 0x597aa0eb4177 <unknown>
#2 0x597aa0f00fba <unknown>
#3 0x597aa0ef46b7 <unknown>
#4 0x597aa0f22442 <unknown>
#5 0x597aa0ef4028 <unknown>
#6 0x597aa0f2260e <unknown>
#7 0x597aa0f4112d <unknown>
#8 0x597aa0f221e3 <unknown>
#9 0x597aa0ef2135 <unknown>
#10 0x597aa0ef313e <unknown>
#11 0x597aa1194e4b <unknown>
#12 0x597aa1198dfa <unknown>
#13 0x597aa11816d5 <unknown>
#14 0x597aa1199a6f <unknown>
#15 0x597aa116569f <unknown>
#16 0x597aa11be098 <unknown>
#17 0x597aa11be262 <unknown>
#18 0x597aa11cff34 <unknown>
#19 0x78936ca7cac3 <unknown>


In [ ]:
data=load('/content/EngSpeech.csv')
place=23
driver=DriverSet()

url=data.iloc[place]["Link"]
driver.get(url)

path=data.iloc[place]["Path"]
titlepath=data.iloc[place]["TitlePath"] if str(data.iloc[place]["TitlePath"])!="nan" else 0
next=data.iloc[place]["Next"] if str(data.iloc[place]["Next"])!="nan" else 0
i=int(data.iloc[place]["Next Page Start"]) if str(data.iloc[place]["Next Page Start"])!="nan" else 0
by=int(data.iloc[place]["By"]) if str(data.iloc[place]["By"])!="nan" else 0
nextbutton=data.iloc[place]["Next Button"] if str(data.iloc[place]["Next Button"])!="nan" else 0
nextlink=data.iloc[place]["Next Page"] if str(data.iloc[place]["Next Page"])!="nan" else 0
nextlinkroot = str(data.iloc[place]["Next Page"])

links = get_onclick_list(By.XPATH, path, nextlinkroot) if str(data.iloc[place]["Checked?"])=="split" else get_link_list(By.XPATH, path)  if titlepath!=0 else 0
titles = get_list(By.XPATH, titlepath)   if str(data.iloc[place]["TitlePath"])!="nan" else 0
jump = 1 if str(data.iloc[place]["Checked?"])=="jump" else 0
click = 1 if str(data.iloc[place]["Checked?"])=="click" else 0
category_links = dict(map(lambda a,j : (a,j) , titles,links)) if str(data.iloc[place]["TitlePath"])!="nan" else {x: get_link_by_text(x) for x in get_list(By.XPATH, path)}
pages=int(data.iloc[place]["Pages"]) if str(data.iloc[place]["Pages"])!="nan" else 0


secondpage=True
while True:
  if str(data.iloc[place]["Checked?"])=="Unresponsive":
    break
  if secondpage==True:
    #print(i)
    if jump!=1 and click!=1 and nextlink!=0: #Go to 2nd page
      print("2nd page"+" "+f'{nextlink}{i}')
      driver.get(f'{nextlink}{i}')
      category_links2 = dict(map(lambda a,j : (a,j) , titles,links)) if str(data.iloc[place]["TitlePath"])!="nan" else {x: get_link_by_text(x) for x in get_list(By.XPATH, path)}
      category_links.update(category_links2)
    #i+=by if by!=0 else 1
    secondpage=False
  try:#See if next page exists
    if jump==1:
      print("hi")
      nexturl= get_link_by_xpath(nextbutton)#next link
      #i+=by if by!=0 else 1
    elif click==1:
      print("hi2")
      driver.find_element(By.XPATH,nextbutton).click()
      #i+=by if by!=0 else 1
      #driver.implicitly_wait(30)
    elif next!=0 or pages!=0 or nextbutton!=0 or nextlink!=0:
      print('hi3')
      if str(next)!="0":
        print('hey1')
        nexturl= driver.find_element(By.PARTIAL_LINK_TEXT,next).get_attribute("href")
      elif str(nextbutton)!="0": #make sure it exists first
        print('hey2')
        if nextlink!="0" or path=="Nan":
          print('hello1')
          nexturl= get_link_by_xpath(nextbutton)
        else:
          print('hello2')
          nexturl= f'{nextlink}{i}'
        #print(nexturl)
      elif str(nextlink)!="0":
        print('hey3')
        if next==0 and nextbutton==0 and i==0 and by==0:
          print('hello1')
          print("No next page")
          break
      #i+=by if by!=0 else 1
      #print(nexturl)
    else:
      print('hi4')
      print("No next page")
      break
    i+=by if by!=0 else 1
  except NoSuchElementException: #If not then
    print("No next page!")
    break
  else: #If Exists, then get the links
    print(i)
    print("h")
    if click!=1:
      driver.get(nexturl)
      print(nexturl)
    time.sleep(5)
    links = get_link_list(By.XPATH, path)  if str(data.iloc[place]["TitlePath"])!="nan" else 0
    titles = get_list(By.XPATH, titlepath)   if str(data.iloc[place]["TitlePath"])!="nan" else 0
    category_links2 = dict(map(lambda a,j : (a,j) , titles,links)) if str(data.iloc[place]["TitlePath"])!="nan" else {x: get_link_by_text(x) for x in get_list(By.XPATH, path)}
    category_links.update(category_links2)
    if i==5:
      break
    if i==pages and pages!=0:
      break

#Hi

In [ ]:
linklist=list(category_links.values())

In [ ]:
data=[]
count=0
for i in linklist:
  article=NewsPlease.from_url(i)
  #append information to dataframe
  data.append([article.title,
               article.authors,
               article.date_publish,
               article.description,
               article.maintext,
               article.language,
               article.url])
  count+=1
  #Make dataframe of Title, Author, Date publish, Description, Main Text, Language, URL
df=pd.DataFrame(data,columns=["Title","Authors","Date Published","Description","Main Text","Language","URL"])
df

,Title,Authors,Date Published,Description,Main Text,Language,URL
0,Tancoo: calls on Imbert to have a heart and in...,[],2021-09-08 22:20:42,Today we come to you to shine some light on a ...,Today we come to you to shine some light on a ...,en,https://unctt.org/tancoo-calls-on-imbert-to-ha...
1,Rowley’s Mad Power Grab in Tobago,[],2021-02-16 16:16:02,"My friends, we are seeing a government that is...","My friends, we are seeing a government that is...",en,https://unctt.org/rowleys-mad-power-grab-in-to...
2,Presentation of UNC general election candidate...,[],2020-07-19 11:52:20,T&T family! UNC family! It is my absolute plea...,T&T family! UNC family! It is my absolute plea...,en,https://unctt.org/presentation-of-unc-general-...
3,Kamla: My vision is to build an intelligent na...,[],2020-06-26 14:42:23,"“ON THE BALLOT” UNC VIRTUAL MEETING (June 25, ...","“ON THE BALLOT” UNC VIRTUAL MEETING (June 25, ...",en,https://unctt.org/kamla-my-vision-is-to-build-...
4,MNF - Kamla: There is hope,[],2020-03-11 16:05:27,Introduction Good evening T&T! Good evening UN...,Introduction\nGood evening T&T! Good evening U...,en,https://unctt.org/mnf-kamla-there-is-hope/
...,...,...,...,...,...,...,...
87,PM Speech at the Opening of the Caribbean Inve...,[About The Author],2013-06-10 17:31:30,ADDRESS BY: The Honourable Kamla Persad-Bisses...,ADDRESS BY:\nThe Honourable Kamla Persad-Bisse...,en,https://unctt.org/pm-speech-at-the-opening-of-...
88,Speaking notes for PM Kamla Persad-Bissessar a...,[About The Author],2013-06-01 20:18:01,Good afternoon .. and welcome. Today has been ...,Good afternoon .. and welcome.\nToday has been...,en,https://unctt.org/speaking-notes-for-pm-kamla-...
89,PM Kamla speech at the US Vice President-CARIC...,[About The Author],2013-05-29 22:52:55,The Honourable Kamla Persad- Bissessar on the ...,The Honourable Kamla Persad- Bissessar on the ...,en,https://unctt.org/pm-kamla-speech-at-the-us-vi...
90,"PM: Foreign or local “I don’t care, investigat...",[About The Author],2013-05-27 17:15:10,“Now they are saying bring somebody from abroa...,“Now they are saying bring somebody from abroa...,en,https://unctt.org/pm-foreign-or-local-i-dont-c...


In [ ]:
from google.colab import files
import csv

In [ ]:
df.to_csv('IDBTT01.csv',sep=",",quotechar='"',index=False,quoting=csv.QUOTE_ALL)

#df.to_csv('IDBTT01.csv')
files.download('IDBTT01.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
article=NewsPlease.from_url("https://www.ttparliament.org/wp-content/uploads/2024/01/hh20240126u.pdf")

In [ ]:
[article.title,
               article.authors,
               article.date_publish,
               article.description,
               article.maintext,
               article.language,
               article.url]

[None,
 [],
 None,
 None,
 None,
 'en',
 'https://www.ttparliament.org/wp-content/uploads/2024/01/hh20240126u.pdf']